In [2]:
import tushare as ts
import pandas as pd
import numpy as np
import os
import time
import seaborn as sns
import lightgbm as lgb
import pickle
from datetime import date, timedelta
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

In [3]:
# 获取沪深300指数当前成分股
ts_api = ts.pro_api('f75ac7a71ba8bf7118966e7fb1a0224dda2345fa0f54d1f7b010b91b')
hs300 = ts_api.index_weight(index_code='000300.SH', start_date='20230604', end_date='20230631')
hs300

,index_code,con_code,trade_date,weight
0,000300.SH,601398.SH,20230630,0.894
1,000300.SH,600048.SH,20230630,0.495
2,000300.SH,603185.SH,20230630,0.081
3,000300.SH,600276.SH,20230630,1.132
4,000300.SH,603899.SH,20230630,0.088
...,...,...,...,...
295,000300.SH,300601.SZ,20230630,0.090
296,000300.SH,601998.SH,20230630,0.097
297,000300.SH,601698.SH,20230630,0.044
298,000300.SH,600893.SH,20230630,0.298


In [4]:
# 以000001.SZ为例搭建XGBoost预测模型
data = ts_api.daily(ts_code='603650.SH')
data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,603650.SH,20230313,37.00,38.45,36.59,36.92,37.20,-0.28,-0.7527,222868.53,835143.047
1,603650.SH,20230310,38.34,40.10,36.70,37.20,38.34,-1.14,-2.9734,381018.48,1466180.246
2,603650.SH,20230309,35.11,38.34,35.11,38.34,34.85,3.49,10.0143,250360.02,933619.829
3,603650.SH,20230308,33.46,35.27,32.69,34.85,33.50,1.35,4.0299,132001.32,449888.191
4,603650.SH,20230307,35.60,36.06,33.41,33.50,35.11,-1.61,-4.5856,105791.00,366451.208
...,...,...,...,...,...,...,...,...,...,...,...
1139,603650.SH,20180703,25.97,25.97,25.97,25.97,23.61,2.36,10.0000,2961.49,7690.990
1140,603650.SH,20180702,23.61,23.61,23.61,23.61,21.46,2.15,10.0200,1789.64,4225.340
1141,603650.SH,20180629,21.46,21.46,21.46,21.46,19.51,1.95,9.9900,661.27,1419.085
1142,603650.SH,20180628,19.51,19.51,19.51,19.51,17.74,1.77,9.9800,363.91,709.988


In [30]:
data = ts_api.daily(ts_code='000688.SZ')
n = 9
open_pct = (100*data['open'].loc[::-1].diff().loc[len(data)-2:].values/data['open'].loc[:1:-1].values).tolist()
high_pct = (100*data['high'].loc[::-1].diff().loc[len(data)-2:].values/data['high'].loc[:1:-1].values).tolist()
low_pct = (100*data['low'].loc[::-1].diff().loc[len(data)-2:].values/data['low'].loc[:1:-1].values).tolist()
close_pct = (100*data['close'].loc[::-1].diff().loc[len(data)-2:].values/data['close'].loc[:1:-1].values).tolist()
vol_pct = (100*data['vol'].loc[::-1].diff().loc[len(data)-2:].values/data['vol'].loc[:1:-1].values).tolist()
amount_pct = (100*data['amount'].loc[::-1].diff().loc[len(data)-2:].values/data['amount'].loc[:1:-1].values).tolist()
X = []
y = []
for i in range(len(open_pct)-n):
    X.append(open_pct[i:i+5] + high_pct[i:i+5] + low_pct[i:i+5] + close_pct[i:i+5] + vol_pct[i:i+5] + amount_pct[i:i+5])
    y.append(close_pct[i+n])
model = lgb.LGBMRegressor(learning_rate=0.01, metric='l1')
model.fit(X,y,eval_metric='l2')
day_data = data
open_pct = (100*day_data['open'].loc[::-1].diff().values[1:]/day_data['open'].loc[::-1].values[:-1]).tolist()[-5:]
high_pct = (100*day_data['high'].loc[::-1].diff().values[1:]/day_data['high'].loc[::-1].values[:-1]).tolist()[-5:]
low_pct = (100*day_data['low'].loc[::-1].diff().values[1:]/day_data['low'].loc[::-1].values[:-1]).tolist()[-5:]
close_pct = (100*day_data['close'].loc[::-1].diff().values[1:]/day_data['close'].loc[::-1].values[:-1]).tolist()[-5:]
vol_pct = (100*day_data['vol'].loc[::-1].diff().values[1:]/day_data['vol'].loc[::-1].values[:-1]).tolist()[-5:]
amount_pct = (100*day_data['amount'].loc[::-1].diff().values[1:]/day_data['amount'].loc[::-1].values[:-1]).tolist()[-5:]
X = [open_pct + high_pct + low_pct + close_pct + vol_pct + amount_pct]
model.predict(X)

array([0.3646896])

In [4]:
for code in tqdm(hs300['con_code']):
    data = ts_api.daily(ts_code=code)
    open_pct = (100*data['open'].loc[::-1].diff().loc[len(data)-2:].values/data['open'].loc[:1:-1].values).tolist()
    high_pct = (100*data['high'].loc[::-1].diff().loc[len(data)-2:].values/data['high'].loc[:1:-1].values).tolist()
    low_pct = (100*data['low'].loc[::-1].diff().loc[len(data)-2:].values/data['low'].loc[:1:-1].values).tolist()
    close_pct = (100*data['close'].loc[::-1].diff().loc[len(data)-2:].values/data['close'].loc[:1:-1].values).tolist()
    vol_pct = (100*data['vol'].loc[::-1].diff().loc[len(data)-2:].values/data['vol'].loc[:1:-1].values).tolist()
    amount_pct = (100*data['amount'].loc[::-1].diff().loc[len(data)-2:].values/data['amount'].loc[:1:-1].values).tolist()
    X = []
    y = []
    for i in range(len(open_pct)-5):
        X.append(open_pct[i:i+5] + high_pct[i:i+5] + low_pct[i:i+5] + close_pct[i:i+5] + vol_pct[i:i+5] + amount_pct[i:i+5])
        y.append(close_pct[i+5])
    model = lgb.LGBMRegressor(learning_rate=0.01, metric='l1', n_estimators=256, num_leaves=64)
    model.fit(X,y,eval_metric='l2')
    filename = 'model/{}.pickle'.format(code)
    pickle.dump(model, open(filename,'wb'))

100%|██████████| 300/300 [21:12<00:00,  4.24s/it]


In [216]:
# Unimodel
X = []
y = []
for code in tqdm(hs300['con_code']):
    data = ts_api.daily(ts_code=code)
    open_pct = (100*data['open'].loc[::-1].diff().loc[len(data)-2:].values/data['open'].loc[:1:-1].values).tolist()
    high_pct = (100*data['high'].loc[::-1].diff().loc[len(data)-2:].values/data['high'].loc[:1:-1].values).tolist()
    low_pct = (100*data['low'].loc[::-1].diff().loc[len(data)-2:].values/data['low'].loc[:1:-1].values).tolist()
    close_pct = (100*data['close'].loc[::-1].diff().loc[len(data)-2:].values/data['close'].loc[:1:-1].values).tolist()
    vol_pct = (100*data['vol'].loc[::-1].diff().loc[len(data)-2:].values/data['vol'].loc[:1:-1].values).tolist()
    amount_pct = (100*data['amount'].loc[::-1].diff().loc[len(data)-2:].values/data['amount'].loc[:1:-1].values).tolist()
    for i in range(len(open_pct)-5):
        X.append(open_pct[i:i+5] + high_pct[i:i+5] + low_pct[i:i+5] + close_pct[i:i+5] + vol_pct[i:i+5] + amount_pct[i:i+5])
        y.append(close_pct[i+5])
    
model = lgb.LGBMRegressor(learning_rate=0.01, metric='l2')
model.fit(X,y,eval_metric='l2')
filename = 'model/{}.pickle'.format('Uniform')
pickle.dump(model, open(filename,'wb'))

100%|██████████| 300/300 [14:45<00:00,  2.95s/it]


In [32]:
codes = ','.join(hs300['con_code'].values)
codes
data = ts_api.daily(ts_code=codes, start_date=str(date.today()-timedelta(8)).replace('-',''), end_date=str(date.today()).replace('-',''))
data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20230707,11.20,11.27,11.18,11.20,11.24,-0.04,-0.3559,484440.83,543212.456
1,000002.SZ,20230707,13.92,14.01,13.81,13.87,13.97,-0.10,-0.7158,394498.13,548059.812
2,000063.SZ,20230707,43.28,43.50,41.98,42.99,43.28,-0.29,-0.6701,681196.34,2916126.790
3,000069.SZ,20230707,4.47,4.52,4.45,4.47,4.48,-0.01,-0.2232,145020.73,64944.476
4,000100.SZ,20230707,4.12,4.26,4.11,4.16,4.10,0.06,1.4634,3972564.53,1658360.886
...,...,...,...,...,...,...,...,...,...,...,...
1795,688363.SH,20230630,88.44,90.28,88.19,89.16,88.66,0.50,0.5640,16111.84,143901.616
1796,688396.SH,20230630,52.61,53.29,52.13,52.41,52.83,-0.42,-0.7950,33960.87,178329.385
1797,688561.SH,20230630,53.03,53.18,51.48,51.81,52.98,-1.17,-2.2084,47133.75,246371.413
1798,688599.SH,20230630,42.10,43.06,41.61,42.61,42.03,0.58,1.3800,147232.19,626437.700


In [33]:
p = {}
for code in tqdm(hs300['con_code']):
    day_data = data[data['ts_code']==code]
    if day_data.shape[0] < 6:
        print(code)
        continue
    open_pct = (100*day_data['open'].loc[::-1].diff().values[1:]/day_data['open'].loc[::-1].values[:-1]).tolist()
    high_pct = (100*day_data['high'].loc[::-1].diff().values[1:]/day_data['high'].loc[::-1].values[:-1]).tolist()
    low_pct = (100*day_data['low'].loc[::-1].diff().values[1:]/day_data['low'].loc[::-1].values[:-1]).tolist()
    close_pct = (100*day_data['close'].loc[::-1].diff().values[1:]/day_data['close'].loc[::-1].values[:-1]).tolist()
    vol_pct = (100*day_data['vol'].loc[::-1].diff().values[1:]/day_data['vol'].loc[::-1].values[:-1]).tolist()
    amount_pct = (100*day_data['amount'].loc[::-1].diff().values[1:]/day_data['amount'].loc[::-1].values[:-1]).tolist()
    X = [open_pct + high_pct + low_pct + close_pct + vol_pct + amount_pct]
    filename = 'model/{}.pickle'.format(code)
    model = pickle.load(open(filename,'rb'))
    p[model.predict(X)[0]] = code

100%|██████████| 300/300 [00:01<00:00, 218.84it/s]


In [34]:
[p[key] for key in sorted(p, reverse=True)[:10]]

['688223.SH',
 '601872.SH',
 '688187.SH',
 '300496.SZ',
 '300450.SZ',
 '600132.SH',
 '300750.SZ',
 '000963.SZ',
 '002459.SZ',
 '601088.SH']

In [35]:
sorted(p, reverse=True)

[2.389265097924882,
 1.7008647526749183,
 1.5171092528809524,
 1.3647159172726357,
 1.3390269886451989,
 1.2929500735955013,
 1.0272354928544432,
 1.0234795226755076,
 0.9509260986428318,
 0.7809604539554078,
 0.7545437215644626,
 0.7144586596615462,
 0.7050885740486637,
 0.6937375977453165,
 0.6936768817838528,
 0.691380118443936,
 0.6814609017273447,
 0.669543817002505,
 0.652253797143198,
 0.6460314633258284,
 0.6415862384832023,
 0.6321267630625426,
 0.6269360583319968,
 0.6165029332696491,
 0.6121334382410432,
 0.6056493477257946,
 0.595258234367268,
 0.5590841713135429,
 0.5364936745948358,
 0.5239880723743693,
 0.49967117020189217,
 0.47487383971182207,
 0.47310438408196165,
 0.46626954049027297,
 0.4643882187906257,
 0.44683929821188917,
 0.44072280787478596,
 0.4339577540076742,
 0.43236034748279445,
 0.4322637261422137,
 0.4278662255872018,
 0.41813280528794416,
 0.3779813631547476,
 0.37650140899602935,
 0.37009509115726,
 0.3587825407227747,
 0.35498398164762834,
 0.3502201

In [36]:
np.mean(list(p.keys()))

-0.0671660417553661

In [37]:
[p[key] for key in sorted(p)[:10]]

['603185.SH',
 '300124.SZ',
 '605117.SH',
 '603799.SH',
 '688111.SH',
 '688005.SH',
 '601615.SH',
 '601698.SH',
 '600958.SH',
 '600989.SH']

# 坚定持有
周一开盘买，周五收盘卖

In [7]:
ts_api = ts.pro_api('f75ac7a71ba8bf7118966e7fb1a0224dda2345fa0f54d1f7b010b91b')
hs300 = ts_api.index_weight(index_code='000300.SH', start_date='20230301', end_date='20230331')
hs300

,index_code,con_code,trade_date,weight
0,000300.SH,603195.SH,20230301,0.0638
1,000300.SH,003816.SZ,20230301,0.1092
2,000300.SH,300782.SZ,20230301,0.1905
3,000300.SH,601615.SH,20230301,0.2032
4,000300.SH,601138.SH,20230301,0.1989
...,...,...,...,...
295,000300.SH,000157.SZ,20230301,0.1698
296,000300.SH,000069.SZ,20230301,0.1107
297,000300.SH,000063.SZ,20230301,0.5270
298,000300.SH,000002.SZ,20230301,0.5894


In [40]:
def idotvince(n):
    def flat(l):
        for item in l:
            if type(item) == list:
                flat(item)
            else:
                r.append(item)
    def sub(n, frac):
        result = []
        if n == 1:
            return [[frac]]
        for i in range(0,frac+1):
            result.append([i] + [ k for k in sub(n-1,frac-i)])
        return result
    r = []
    flat(sub(n,100))
    return np.array(r).reshape((-1,n))
idotvince(3)

array([[  0,   0, 100],
       [  0,   1,  99],
       [  0,   2,  98],
       ...,
       [ 99,   0,   1],
       [ 99,   1,   0],
       [100,   0,   0]])

In [49]:
def idiot_vince(n, frac):
    # return all combinations of n ints sum up to frac
    result = []
    if n == 1:
        return [[frac]]
    for i in range(0,frac+1):
        result+=[[i] + k for k in idiot_vince(n-1,frac-i)]
    return result
idiot_vince(4,100)

[[0, 0, 0, 100],
 [0, 0, 1, 99],
 [0, 0, 2, 98],
 [0, 0, 3, 97],
 [0, 0, 4, 96],
 [0, 0, 5, 95],
 [0, 0, 6, 94],
 [0, 0, 7, 93],
 [0, 0, 8, 92],
 [0, 0, 9, 91],
 [0, 0, 10, 90],
 [0, 0, 11, 89],
 [0, 0, 12, 88],
 [0, 0, 13, 87],
 [0, 0, 14, 86],
 [0, 0, 15, 85],
 [0, 0, 16, 84],
 [0, 0, 17, 83],
 [0, 0, 18, 82],
 [0, 0, 19, 81],
 [0, 0, 20, 80],
 [0, 0, 21, 79],
 [0, 0, 22, 78],
 [0, 0, 23, 77],
 [0, 0, 24, 76],
 [0, 0, 25, 75],
 [0, 0, 26, 74],
 [0, 0, 27, 73],
 [0, 0, 28, 72],
 [0, 0, 29, 71],
 [0, 0, 30, 70],
 [0, 0, 31, 69],
 [0, 0, 32, 68],
 [0, 0, 33, 67],
 [0, 0, 34, 66],
 [0, 0, 35, 65],
 [0, 0, 36, 64],
 [0, 0, 37, 63],
 [0, 0, 38, 62],
 [0, 0, 39, 61],
 [0, 0, 40, 60],
 [0, 0, 41, 59],
 [0, 0, 42, 58],
 [0, 0, 43, 57],
 [0, 0, 44, 56],
 [0, 0, 45, 55],
 [0, 0, 46, 54],
 [0, 0, 47, 53],
 [0, 0, 48, 52],
 [0, 0, 49, 51],
 [0, 0, 50, 50],
 [0, 0, 51, 49],
 [0, 0, 52, 48],
 [0, 0, 53, 47],
 [0, 0, 54, 46],
 [0, 0, 55, 45],
 [0, 0, 56, 44],
 [0, 0, 57, 43],
 [0, 0, 58, 42],
 [0, 0